# Технический скрипт для домашки 2

Скачиваем данные о ценных бумагах. 

```
pip install yfinance
```

In [2]:
import pandas as pd
import re

# компании из индекса 
df = pd.read_html('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
tickers_sp = list(df.Symbol.values)

In [14]:
df2 = pd.read_html('https://en.wikipedia.org/wiki/NASDAQ-100')[4]
tickers_nasdaq = list(df2.Ticker.values)

In [15]:
df3 = pd.read_html('https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average')[1]
tickers_dji = list(df3.Symbol.str.split(':').apply(lambda w: re.sub('\xa0', '', w[-1])).values)

In [16]:
tickers = list(set(tickers_sp + tickers_nasdaq + tickers_dji))
len(tickers)

523

In [17]:
import yfinance as yf
import datetime

start = datetime.datetime(2013,3,5)
end = datetime.datetime(2023,3,5)
data = yf.download(tickers, start=start, end=end)

[*********************100%***********************]  523 of 523 completed

2 Failed downloads:
- BF.B: No data found for this date range, symbol may be delisted
- BRK.B: No timezone found, symbol may be delisted


In [18]:
data = data['Adj Close']
data.tail()

,A,AAL,AAP,AAPL,ABBV,ABC,ABNB,ABT,ACGL,ACN,...,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZM,ZS,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2023-02-27 00:00:00,142.179993,15.630000,140.630005,147.919998,154.279999,156.630005,123.529999,99.769997,69.209999,266.250000,...,110.550003,34.549999,102.550003,126.570000,123.300003,296.230011,50.389999,73.720001,131.839996,165.470001
2023-02-28 00:00:00,141.970001,15.980000,144.960007,147.410004,153.899994,155.559998,123.279999,101.720001,70.000000,265.549988,...,109.910004,38.070000,102.650002,127.160004,123.870003,300.250000,50.619999,74.589996,131.149994,167.000000
2023-03-01 00:00:00,137.509995,16.030001,138.820007,145.309998,155.270004,156.509995,120.830002,100.830002,70.970001,263.589996,...,110.889999,37.360001,101.089996,126.199997,122.080002,302.339996,50.439999,69.620003,128.440002,167.570007
2023-03-02 00:00:00,141.669998,16.120001,139.210007,145.910004,154.380005,154.899994,122.379997,102.629997,70.900002,265.260010,...,111.400002,37.680000,102.250000,128.679993,122.989998,306.059998,48.310001,70.629997,134.130005,168.580002
2023-03-03 00:00:00,143.929993,16.360001,138.479996,151.029999,156.059998,156.149994,125.730003,104.449997,71.300003,269.200012,...,112.809998,38.740002,103.919998,129.669998,126.220001,309.450012,49.230000,70.809998,119.239998,170.559998


In [19]:
# избавимся от пустых тикеров
isnull = data.isnull().sum()/data.shape[0]
bad_tickers = list(isnull[isnull == 1].index)
data.drop(bad_tickers, axis=1, inplace=True)

In [20]:
data.isnull().sum().sort_values()[-50:]/data.shape[0]

ENPH    0.000000
ESS     0.000000
IQV     0.018268
NWSA    0.029388
NWS     0.029388
CDW     0.031771
ALLE    0.071485
HLT     0.078237
PAYC    0.111597
JD      0.121922
ANET    0.125894
CTLT    0.140985
SYF     0.140985
CZR     0.155282
CFG     0.156076
KEYS    0.163225
QRVO    0.183479
SEDG    0.206116
ETSY    0.211676
WRK     0.230739
KHC     0.233519
PYPL    0.233519
HPE     0.262907
TEAM    0.277204
FTV     0.333598
HWM     0.366958
LW      0.369738
INVH    0.391581
IR      0.419380
VICI    0.483320
ZS      0.503574
CDAY    0.514694
PDD     0.539714
MRNA    0.576648
FOXA    0.601668
FOX     0.602065
DOW     0.604051
ZM      0.612391
CTVA    0.622319
CRWD    0.627085
DDOG    0.654488
CARR    0.704130
OTIS    0.704130
LCID    0.754567
ABNB    0.777601
OGN     0.819698
GFS     0.865766
RIVN    0.869341
CEG     0.888006
GEHC    0.978952
dtype: float64

In [21]:
data.to_csv('price.tsv', sep='\t')

In [ ]:
df['S&P-500'] = True
df.set_index('Symbol', inplace=True)

df2['NASDAQ'] = True
df2.set_index('Ticker', inplace=True)

In [29]:
df = df.join(df2[['NASDAQ']], how='outer')

In [30]:
df3['DJI'] = True
df3['Symbol'] = df3['Symbol'].apply(lambda w: re.sub('\xa0', '', w.split(':')[-1]))
df4 = df3[['Symbol', 'DJI', 'Company']].set_index('Symbol')

df = df.join(df4, how='outer', rsuffix='dj')

In [31]:
df.to_csv('information.tsv', sep='\t')

In [32]:
df.shape

(523, 11)

In [33]:
df3.columns

Index(['Company', 'Exchange', 'Symbol', 'Industry', 'Date added', 'Notes',
       'Index weighting', 'DJI'],
      dtype='object')